In [1]:
# Импорт библиотеки Pandas
import pandas as pd

## Снимаем ограничение на количество отображаемых колонок.
---

In [2]:
pd.set_option("display.max_rows", None)

In [3]:
# Загрузка CSV-файла в объект DataFrame.
# Метод pd.read_csv считывает данные из файла CSV (формата .csv) и загружает в таблицу.
df_games = pd.read_csv("games.csv")

# Отображаем первые 5 строк датафрейма
df_games.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


# Перед анализом данных, очень важно выполнить этап предварительной обработки и очистки данных **(data cleaning & preprocessing)**. Это фундаментальный шаг, обеспечивающий корректность последующего анализа и визуализации.
---

## Краткий чек-лист перед анализом:

| Шаг | Действие |
| ----------- | ----------- |
| 1. Удаление пробелов. | Удалить лишние пробелы в названиях столбцов. |
| 2. Обработка **Platform**. | Заменить пропущенные значения на `"Unknown"`, привести к верхнему регистру. |
| 3. Обработка **Year_of_Release**. | Привести к целому типу `Int64`. |
| 4. Обработка **Genre**. | Заменить пропуски на `"Unknown"`. |
| 5. Обработка продаж. | Заполнить пропуски в `NA/EU/JP/Other_sales` нулями `(0.0)`. |
| 6. Обработка **Rating**. | Заменить `NaN` на `"Unknown"`, привести к верхнему регистру. |
---

In [4]:
# 1. Удалить лишние пробелы в названиях столбцов.
df_games.columns = df_games.columns.str.strip()

df_games.columns

Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'NA_sales', 'EU_sales',
       'JP_sales', 'Other_sales', 'Critic_Score', 'User_Score', 'Rating'],
      dtype='object')

In [5]:
# 2. Заменить пропущенные значения на "Unknown" в столбце "Platform", привести к верхнему регистру.
df_games["Platform"] = df_games["Platform"].fillna("Unknown").str.upper()

# Отображаем только колонку "Platform" для проверки изменений
df_games["Platform"].head()

0    WII
1    NES
2    WII
3    WII
4     GB
Name: Platform, dtype: object

In [6]:
# 3. Привести к целому типу Int64 столбец Year_of_Release.
df_games["Year_of_Release"] = (
    pd.to_numeric(df_games["Year_of_Release"], errors = "coerce")
    .fillna(0)
    .astype("Int64")
    )

# Отображаем только колонку "Year_of_Release" для проверки изменений
df_games["Year_of_Release"].head()

0    2006
1    1985
2    2008
3    2009
4    1996
Name: Year_of_Release, dtype: Int64

In [7]:
# 4. Заменить пропуски на "Unknown" в столбце Genre.
df_games["Genre"] = df_games["Genre"].fillna("Unknown")

# Отображаем только колонку "Genre" для проверки изменений
df_games["Genre"].head()

0          Sports
1        Platform
2          Racing
3          Sports
4    Role-Playing
Name: Genre, dtype: object

In [8]:
# 5. Заполнить пропуски в `NA/EU/JP/Other_sales` нулями `(0.0)`.
sales_columns_v1 = ["NA_sales", "EU_sales", "JP_sales", "Other_sales"]

for columns in sales_columns_v1:
    df_games[columns] = df_games[columns].fillna(0.0)

# Отображаем колонки продаж для проверки изменений
df_games[sales_columns_v1].head()

,NA_sales,EU_sales,JP_sales,Other_sales
0,41.36,28.96,3.77,8.45
1,29.08,3.58,6.81,0.77
2,15.68,12.76,3.79,3.29
3,15.61,10.93,3.28,2.95
4,11.27,8.89,10.22,1.00


In [9]:
# 6. Заменить "NaN" на "Unknown" в колонке Rating, привести к верхнему регистру.
df_games["Rating"] = df_games["Rating"].fillna("Unknown").str.upper()

# Отображаем только колонку "Rating" для проверки изменений
df_games["Rating"].head()

0          E
1    UNKNOWN
2          E
3          E
4    UNKNOWN
Name: Rating, dtype: object

## **ЗАДАНИЕ 1: ГРУППИРОВКА ПО ЖАНРАМ**
### Задание: Сгруппируй данные по жанрам `(Genre)` и вычисли суммарные продажи в Северной Америке `(NA_sales)` для каждого жанра. Выведи результат.
### Подсказка: Используй метод `groupby()` и агрегирующую функцию `sum()`.
---

In [10]:
grouped_genres = (
    df_games.groupby("Genre")["NA_sales"] # Группируем по жанрам
    .agg("sum")                           # Суммируем продажи в Северной Америке
    .reset_index(name = "Total_NA_Sales") # Сбрасываем индекс и переименовываем столбец
    .sort_values(by = "Total_NA_Sales", ascending = False) # Сортируем по убыванию продаж
)

# Отображаем сгруппированные данные по жанрам
grouped_genres.reset_index(drop = True, inplace = True) # Сбрасываем индекс для лучшего отображения
grouped_genres.head()

,Genre,Total_NA_Sales
0,Action,879.01
1,Sports,684.43
2,Shooter,592.24
3,Platform,445.50
4,Misc,407.27


## **ЗАДАНИЕ 2: СРЕДНИЕ ПРОДАЖИ ПО ПЛАТФОРМАМ**
### Задание: Найди средние продажи по регионам `(NA_sales, EU_sales, JP_sales, Other_sales)` для каждой платформы `(Platform)`. Выведи результат.
### Подсказка: Можно использовать метод `agg()` для вычисления нескольких агрегатов сразу.
---

In [11]:
average_sales = (
    df_games
    .groupby("Platform")
    .agg({"NA_sales": "mean", "EU_sales": "mean", "JP_sales": "mean", "Other_sales": "mean"})
    .reset_index()
    .rename(columns = {
        "NA_sales": "Average_NA_Sales",
        "EU_sales": "Average_EU_Sales",
        "JP_sales": "Average_JP_Sales",
        "Other_sales": "Average_Other_Sales"
    })
)

average_sales.head()

,Platform,Average_NA_Sales,Average_EU_Sales,Average_JP_Sales,Average_Other_Sales
0,2600,0.681203,0.041128,0.000000,0.006842
1,3DO,0.000000,0.000000,0.033333,0.000000
2,3DS,0.160558,0.118231,0.193596,0.025692
3,DC,0.104423,0.032500,0.164615,0.005192
4,DS,0.177778,0.087815,0.081623,0.027550


## **ЗАДАНИЕ 3: СЛИЯНИЕ ДАННЫХ**
### Задание: Представь, что у тебя есть дополнительная таблица с информацией о рейтингах игр, и ты хочешь объединить её с основным датасетом. Создай небольшой `DataFrame` с колонками `Name` и `Critic_Score`, затем объедини его с оригинальным `DataFrame` по столбцу `Name`.
### Подсказка: Используй метод `merge()` для объединения таблиц.
---

In [12]:
# Создание датафрейма с именами игр и оценками критиков.
df_name_critic = pd.DataFrame({
    "Name": ["Call of Duty Black Ops", "The Legend of Zelda: Breath of the Wild", "The Witcher 3: Wild Hunt"],
    "Critic_Score": [10, 10, 9]
})

df_merged = pd.merge(df_games, df_name_critic, how = "left", on = "Name")

# Отображаем объединенный датафрейм
df_merged.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score_x,User_Score,Rating,Critic_Score_y
0,Wii Sports,WII,2006,Sports,41.36,28.96,3.77,8.45,76.0,8,E,NaN
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,UNKNOWN,NaN
2,Mario Kart Wii,WII,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,NaN
3,Wii Sports Resort,WII,2009,Sports,15.61,10.93,3.28,2.95,80.0,8,E,NaN
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,UNKNOWN,NaN


## **ЗАДАНИЕ 4: СОЗДАНИЕ СВОДНОЙ ТАБЛИЦЫ**
### Задание: Создай сводную таблицу, которая покажет средние продажи в Северной Америке и Европе для каждой платформы и жанра. Выведи результат.
### Подсказка: Для создания сводной таблицы используй метод `pivot_table()`, пустые значения можно заменить с помощью параметра `fill_value=0`.
---

In [ ]:
# Синтаксис сводной таблицы:

# df.pivot_table(
#     values ='что_агрегировать',
#     index ='по_чему_группировать_в_строках',
#     columns ='по_чему_группировать_в_столбцах',
#     aggfunc ='с какой функцией агрегировать',
#     fill_value = 0  # чем заменить пропущенные значения (по желанию)
# )

df_pivot_table = df_games.pivot_table(
    values = ["NA_sales", "EU_sales"], # Значения для агрегации
    index = "Platform",                # Индекс для строк
    columns = "Genre",                 # Колонки для столбцов
    aggfunc = "mean",                  # Функция агрегации
    fill_value = 0                     # Заполнение пропусков нулями
)

df_pivot_table.head()

EU_sales                                                    \
Genre       Action Adventure  Fighting      Misc  Platform    Puzzle   
Platform                                                               
2600      0.026885  0.050000  0.035000  0.040000  0.080000  0.078182   
3DO       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3DS       0.068351  0.020789  0.157857  0.044074  0.375357  0.107500   
DC        0.076667  0.021818  0.000000  0.000000  0.305000  0.000000   
DS        0.076206  0.041975  0.008056  0.089412  0.202088  0.108475   

                                                      ...  NA_sales            \
Genre       Racing Role-Playing   Shooter Simulation  ...      Misc  Platform   
Platform                                              ...                       
2600      0.026667     0.000000  0.062917   0.020000  ...  0.668000  1.375556   
3DO       0.000000     0.000000  0.000000   0.000000  ...  0.000000  0.000000   
3DS       0.426364     0.169432  0.050000   0.261935  ...  0.049815  0.473929   
DC        0.085000     0.000000  0.000000   0.000000  ...  0.000000  0.630000   
DS        0.146418     0.087350  0.012143   0.126537  ...  0.137673  0.405604   

                                                                          \
Genre       Puzzle    Racing Role-Playing   Shooter Simulation    Sports   
Platform                                                                   
2600      1.243636  0.456667       0.0000  1.028333   0.420000  0.268333   
3DO       0.000000  0.000000       0.0000  0.000000   0.000000  0.000000   
3DS       0.065000  0.577273       0.2700  0.094286   0.244516  0.063846   
DC        0.000000  0.183333       0.0000  0.000000   0.000000  0.214000   
DS        0.146992  0.306567       0.2307  0.153810   0.238233  0.104595   

                            
Genre     Strategy Unknown  
Platform                    
2600      0.000000     0.0  
3DO       0.000000     0.0  
3DS       0.052000     0.0  
DC        0.000000     0.0  
DS        0.109747     0.0  

[5 rows x 26 columns]

## **ЗАДАНИЕ 5: ГРУППИРОВКА ДАННЫХ ПО ПЛАТФОРМАМ И ЖАНРАМ**
### Задание: Сгруппируй данные по платформам `(Platform)` и жанрам `(Genre)` и найди средние продажи в Японии `(JP_sales)`.
### Подсказка: Используй метод `groupby()` и функцию `mean()`.
---

In [31]:
df_jp_sales = (
    df_games
    .groupby(["Platform", "Genre"])["JP_sales"]
    .mean()
    .reset_index()
)

df_jp_sales.tail()

,Platform,Genre,JP_sales
289,XONE,Role-Playing,0.000714
290,XONE,Shooter,0.004211
291,XONE,Simulation,0.000000
292,XONE,Sports,0.000526
293,XONE,Strategy,0.000000


## **ЗАДАНИЕ 6: СОЗДАНИЕ СВОДНОЙ ТАБЛИЦЫ С МАКСИМАЛЬНЫМИ ПРОДАЖАМИ**
### Задание: Создай сводную таблицу, которая покажет максимальные продажи в Северной Америке `(NA_sales)` для каждой платформы и жанра. Выведи результат.
### Подсказка: Используй `pivot_table()` с агрегирующей функцией `max()`.
---

In [30]:
# Синтаксис сводной таблицы:

# df.pivot_table(
#     values ='что_агрегировать',
#     index ='по_чему_группировать_в_строках',
#     columns ='по_чему_группировать_в_столбцах',
#     aggfunc ='с какой функцией агрегировать',
#     fill_value = 0  # чем заменить пропущенные значения (по желанию)
# )

df_na_sales_max = df_games.pivot_table(
    values = "NA_sales",
    index = "Platform",
    columns = "Genre",
    aggfunc = "max",
    fill_value = 0
)

df_na_sales_max.head()

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy,Unknown
Platform,,,,,,,,,,,,,
2600,2.06,1.21,0.72,1.07,4.21,7.28,1.10,0.00,4.00,0.42,0.52,0.00,0.0
3DO,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
3DS,2.09,0.18,3.27,0.62,4.89,0.32,5.03,5.28,0.48,2.03,0.36,0.25,0.0
DC,0.41,0.52,0.00,0.00,1.26,0.00,1.10,0.00,0.00,0.00,1.12,0.00,0.0
DS,4.34,1.87,0.36,4.74,11.28,3.43,9.71,6.38,0.95,9.05,1.63,1.02,0.0


## **ЗАДАНИЕ 7: СВОДНАЯ ТАБЛИЦА ПО ГОДАМ**
### Задание: Создай сводную таблицу, которая покажет суммарные продажи в Японии `(JP_sales)` для каждой платформы по годам выпуска игр `(Year_of_Release)`. Выведи результат.
### Подсказка: В качестве индекса укажи платформу, а в качестве колонок — год выпуска игры, пустые значения можно заменить с помощью параметра `fill_value=0ЗУрок №30. Обработка данных с Pandas. Часть 2`.
---

In [35]:
# Синтаксис сводной таблицы:

# df.pivot_table(
#     values ='что_агрегировать',
#     index ='по_чему_группировать_в_строках',
#     columns ='по_чему_группировать_в_столбцах',
#     aggfunc ='с какой функцией агрегировать',
#     fill_value = 0  # чем заменить пропущенные значения (по желанию)
# )

df_jp_sales_sum = df_games.pivot_table(
    values = "JP_sales",
    index = "Platform",
    columns = "Year_of_Release",
    aggfunc = "sum",
    fill_value = 0
)

df_jp_sales_sum.head()

Year_of_Release,0,1980,1981,1982,1983,1984,1985,1986,1987,1988,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Platform,,,,,,,,,,,,,,,,,,,,,
2600,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3DO,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3DS,0.05,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,12.83,19.98,23.57,20.69,15.52,8.03
DC,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.02,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
DS,0.55,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,...,26.81,24.01,24.21,19.86,4.32,3.72,0.00,0.00,0.00,0.00
